In [30]:
import pandas as pd
import numpy as np

from dsgd.DSClassifierMultiQ import DSClassifierMultiQ

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

In [2]:
df = pd.read_csv("https://huggingface.co/datasets/furrutiav/sac_nllf/raw/main/train.csv")
df = df.drop(columns=['index'])
print("Dataframe shape: ", df.shape)
df.head()
test = pd.read_csv("https://huggingface.co/datasets/furrutiav/sac_nllf/raw/main/val.csv")
test = test.drop(columns=['index'])
print("Dataframe shape: ", test.shape)

Dataframe shape:  (1400, 27)
Dataframe shape:  (200, 27)


In [3]:
y = df['label']
X = df.drop(columns=['label'])
y = y.to_numpy()
X = X.to_numpy()    
y_test = test['label']
X_test = test.drop(columns=['label'])
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [34]:
DSC2 = DSClassifierMultiQ(2, min_iter=20, max_iter=1000, debug_mode=True, lr=0.1, batch_size=4000,
                                lossfn="MSE", num_workers=1, min_dloss=1e-5, precompute_rules=True)

losses, epoch, dt = DSC2.fit(X, y, add_single_rules=True,
                            single_rules_breaks=7, add_mult_rules=False,
                                column_names=df.columns[:-1], print_every_epochs=1, print_final_model=False)
y_pred2 = DSC2.predict(X_test)


Optimization started
Processing epoch	1	1.0000	

Processing epoch	23	0.1932	
Training time: 9.00s, epochs: 23

Least training loss reached: 0.194


In [35]:
acc2 = accuracy_score(y_test, y_pred2)
print("Accuracy: ", acc2)
print(confusion_matrix(y_test, y_pred2))
print(classification_report(y_test, y_pred2))

Accuracy:  0.695
[[78 34]
 [27 61]]
              precision    recall  f1-score   support

           0       0.74      0.70      0.72       112
           1       0.64      0.69      0.67        88

    accuracy                           0.69       200
   macro avg       0.69      0.69      0.69       200
weighted avg       0.70      0.69      0.70       200



In [25]:
DSC = DSClassifierMultiQ(2, min_iter=20, max_iter=1000, debug_mode=True, lr=0.1, batch_size=3000,
                                lossfn="MSE", num_workers=1, min_dloss=1e-6, precompute_rules=True)

losses, epoch, dt = DSC.fit(X, y, add_single_rules=True,
                            single_rules_breaks=3, add_mult_rules=False,
                                column_names=df.columns[:-1], print_every_epochs=1, print_final_model=False)
y_pred = DSC.predict(X_test)
acc = accuracy_score(y_test, y_pred)

Optimization started
Processing epoch	23	0.2033	
Training time: 8.91s, epochs: 23

Least training loss reached: 0.206


In [31]:
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("F1 Score: ", f1_score(y_test, y_pred, average='weighted'))
print("f1 Score macro: ", f1_score(y_test, y_pred, average='macro'))
print("f1 Score micro: ", f1_score(y_test, y_pred, average='micro'))

Accuracy:  0.72
[[83 29]
 [27 61]]
              precision    recall  f1-score   support

           0       0.75      0.74      0.75       112
           1       0.68      0.69      0.69        88

    accuracy                           0.72       200
   macro avg       0.72      0.72      0.72       200
weighted avg       0.72      0.72      0.72       200

F1 Score:  0.7203117724466039
f1 Score macro:  0.716570503087357
f1 Score micro:  0.72


In [28]:
DSC.print_most_important_rules(classes=["0",  "1"])



Most important rules for class 0

	[0.731] R34: 0.018 < b5(N) < 0.084
			0: 0.731	1: 0.000	Unc: 0.269

	[0.657] R5: 0.721 < b2(Y) < 0.894
			0: 0.657	1: 0.000	Unc: 0.343

	[0.456] R94: 0.325 < b11(Y) < 0.606
			0: 0.456	1: 0.000	Unc: 0.544

	[0.431] R67: b1(N) > 0.282
			0: 0.382	1: 0.104	Unc: 0.514

	[0.424] R48: b8(N) < 0.146
			0: 0.394	1: 0.062	Unc: 0.544

	[0.414] R68: b1(Y) < 0.717
			0: 0.379	1: 0.073	Unc: 0.548

	[0.408] R10: 0.104 < b7(N) < 0.229
			0: 0.371	1: 0.078	Unc: 0.551

	[0.404] R20: b6(Y) < 0.600
			0: 0.378	1: 0.054	Unc: 0.568

	[0.400] R18: 0.201 < b6(N) < 0.435
			0: 0.352	1: 0.103	Unc: 0.545

	[0.382] R86: 0.248 < b12(Y) < 0.510
			0: 0.365	1: 0.035	Unc: 0.600

	[0.364] R19: b6(N) > 0.435
			0: 0.352	1: 0.025	Unc: 0.623

	[0.315] R58: 0.008 < b4(N) < 0.047
			0: 0.235	1: 0.187	Unc: 0.578

	[0.287] R85: -0.014 < b12(Y) < 0.248
			0: 0.236	1: 0.112	Unc: 0.652

	[0.286] R3: b2(N) > 0.262
			0: 0.286	1: 0.000	Unc: 0.714

	[0.284] R12: b7(Y) < 0.715
			0: 0.266	1: 0

In [48]:
DSC3 = DSClassifierMultiQ(2, min_iter=50, max_iter=1000, debug_mode=True, lr=0.1, batch_size=4000,
                         lossfn="MSE", num_workers=1, min_dloss=1e-7, precompute_rules=True)

losses, epoch, dt = DSC3.fit(X, y, add_single_rules=True,
                            single_rules_breaks=3, add_mult_rules=False,
                                column_names=df.columns[:-1], print_every_epochs=1, print_final_model=False)

Optimization started
Processing epoch	54	0.2003	
Training time: 21.17s, epochs: 54

Least training loss reached: 0.200


In [49]:
acc3 = accuracy_score(y_test, DSC3.predict(X_test))
print("Accuracy: ", acc3)
print(confusion_matrix(y_test, DSC3.predict(X_test)))
print(classification_report(y_test, DSC3.predict(X_test)))
print("F1 Score: ", f1_score(y_test, DSC3.predict(X_test), average='weighted'))
print("f1 Score macro: ", f1_score(y_test, DSC3.predict(X_test), average='macro'))
print("f1 Score micro: ", f1_score(y_test, DSC3.predict(X_test), average='micro'))

Accuracy:  0.71
[[80 32]
 [26 62]]
              precision    recall  f1-score   support

           0       0.75      0.71      0.73       112
           1       0.66      0.70      0.68        88

    accuracy                           0.71       200
   macro avg       0.71      0.71      0.71       200
weighted avg       0.71      0.71      0.71       200

F1 Score:  0.7107893940921464
f1 Score macro:  0.7076318177235609
f1 Score micro:  0.71
